In [1]:
import os
import pandas as pd
import re
import numpy as np

In [2]:
# read all the crawled data in the same directory
os.chdir("D:\Projects\LEGO\Rebrickable")
files = os.listdir()

In [3]:
# transfer into dataframes
df_lst = [pd.read_csv(file) for file in files]
filenames = [file.split(".")[0] for file in files]
df_dic = dict(zip(filenames,df_lst))

D:\conda\lib\site-packages\IPython\core\interactiveshell.py:3214: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  if (yield from self.run_code(code, result)):


In [4]:
def set_idchange(id):
    id = id.split("-")[0]
    return '"{}"'.format(id)

### Color

In [5]:
colors = df_dic['colors']
colors.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 179 entries, 0 to 178
Data columns (total 4 columns):
id          179 non-null int64
name        179 non-null object
rgb         179 non-null object
is_trans    179 non-null object
dtypes: int64(1), object(3)
memory usage: 5.7+ KB


In [6]:
colors.head()

,id,name,rgb,is_trans
0,-1,[Unknown],0033B2,f
1,0,Black,05131D,f
2,1,Blue,0055BF,f
3,2,Green,237841,f
4,3,Dark Turquoise,008F9B,f


### Inventories

In [7]:
inventories = df_dic['inventories']
inventories.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15046 entries, 0 to 15045
Data columns (total 3 columns):
id         15046 non-null int64
version    15046 non-null int64
set_num    15046 non-null object
dtypes: int64(2), object(1)
memory usage: 352.7+ KB


In [8]:
inventories.set_num = inventories.set_num.apply(set_idchange)

In [9]:
inventories.head()

,id,version,set_num
0,1,1,"""7922"""
1,3,1,"""3931"""
2,4,1,"""6942"""
3,15,1,"""5158"""
4,16,1,"""903"""


In [10]:
inv_parts = df_dic['inventory_parts']
inv_parts.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 755483 entries, 0 to 755482
Data columns (total 5 columns):
inventory_id    755483 non-null int64
part_num        755483 non-null object
color_id        755483 non-null int64
quantity        755483 non-null int64
is_spare        755483 non-null object
dtypes: int64(3), object(2)
memory usage: 28.8+ MB


In [11]:
inv_parts.head()

,inventory_id,part_num,color_id,quantity,is_spare
0,1,48379c01,72,1,f
1,1,48395,7,1,f
2,1,mcsport6,25,1,f
3,1,paddle,0,1,f
4,3,11816pr0005,78,1,f


#### join color

In [12]:
# inv_parts + color
inv_parts2 = inv_parts.merge(colors, how='left',left_on='color_id',right_on='id')

In [13]:
inv_parts2.columns = ['inventory_id', 'part_num', 'color_id', 'quantity', 'is_spare', 'id','color_name', 'rgb', 'is_trans']

In [14]:
inv_parts2 = inv_parts2.drop('id',axis=1)
inv_parts2.head()

,inventory_id,part_num,color_id,quantity,is_spare,color_name,rgb,is_trans
0,1,48379c01,72,1,f,Dark Bluish Gray,6C6E68,f
1,1,48395,7,1,f,Light Gray,9BA19D,f
2,1,mcsport6,25,1,f,Orange,FE8A18,f
3,1,paddle,0,1,f,Black,05131D,f
4,3,11816pr0005,78,1,f,Light Flesh,F6D7B3,f


In [15]:
inv_parts2.to_csv("inv_parts2.csv",index=False)

In [17]:
inv_sets = df_dic['inventory_sets']
inv_sets.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3238 entries, 0 to 3237
Data columns (total 3 columns):
inventory_id    3238 non-null int64
set_num         3238 non-null object
quantity        3238 non-null int64
dtypes: int64(2), object(1)
memory usage: 76.0+ KB


In [18]:
inv_sets.set_num = inv_sets.set_num.apply(set_idchange)

In [19]:
inv_sets.head()

,inventory_id,set_num,quantity
0,35,"""75911""",1
1,35,"""75912""",1
2,39,"""75048""",1
3,39,"""75053""",1
4,50,"""4515""",1


#### Join sets and parts

In [20]:
# inv_parts2 + inventories
inv_all = inv_parts2.merge(inventories.drop("version",axis=1),left_on="inventory_id",right_on="id",how="left")

In [21]:
inv_all = inv_all.drop("id",axis=1)
inv_all.head()

,inventory_id,part_num,color_id,quantity,is_spare,color_name,rgb,is_trans,set_num
0,1,48379c01,72,1,f,Dark Bluish Gray,6C6E68,f,"""7922"""
1,1,48395,7,1,f,Light Gray,9BA19D,f,"""7922"""
2,1,mcsport6,25,1,f,Orange,FE8A18,f,"""7922"""
3,1,paddle,0,1,f,Black,05131D,f,"""7922"""
4,3,11816pr0005,78,1,f,Light Flesh,F6D7B3,f,"""3931"""


In [60]:
# inv_all + sets
inv_all2 = inv_all.merge(set_pth2,on="set_num",how="left")

In [61]:
inv_all2 = inv_all2.drop(["rating"],axis=1)
inv_all2.head()

,inventory_id,part_num,color_id,quantity,is_spare,color_name,rgb,is_trans,set_num,set_name,year,theme_id,num_parts,price_adjust,pcs_num,price_per_pcs,theme_name,parent_theme
0,1,48379c01,72,1,f,Dark Bluish Gray,6C6E68,f,"""7922""",McDonald's Sports Set Number 6 - Orange Vest S...,2004,460,4,NaN,NaN,NaN,Gravity Games,Sports
1,1,48395,7,1,f,Light Gray,9BA19D,f,"""7922""",McDonald's Sports Set Number 6 - Orange Vest S...,2004,460,4,NaN,NaN,NaN,Gravity Games,Sports
2,1,mcsport6,25,1,f,Orange,FE8A18,f,"""7922""",McDonald's Sports Set Number 6 - Orange Vest S...,2004,460,4,NaN,NaN,NaN,Gravity Games,Sports
3,1,paddle,0,1,f,Black,05131D,f,"""7922""",McDonald's Sports Set Number 6 - Orange Vest S...,2004,460,4,NaN,NaN,NaN,Gravity Games,Sports
4,3,11816pr0005,78,1,f,Light Flesh,F6D7B3,f,"""3931""",Emma's Splash Pool,2012,494,43,6.63,43.0,0.15,Friends,Friends


In [62]:
inv_all2.to_csv("inv_all.csv",index=False)

### Parts

In [24]:
part_cat = df_dic['part_categories']
part_cat.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 64 entries, 0 to 63
Data columns (total 2 columns):
id      64 non-null int64
name    64 non-null object
dtypes: int64(1), object(1)
memory usage: 1.1+ KB


In [25]:
part_cat.head()

,id,name
0,1,Baseplates
1,3,Bricks Sloped
2,4,"Duplo, Quatro and Primo"
3,5,Bricks Special
4,6,Bricks Wedged


In [26]:
part_relation = df_dic['part_relationships']
part_relation.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18087 entries, 0 to 18086
Data columns (total 3 columns):
rel_type           18087 non-null object
child_part_num     18087 non-null object
parent_part_num    18087 non-null object
dtypes: object(3)
memory usage: 424.0+ KB


In [27]:
part_relation.head()

,rel_type,child_part_num,parent_part_num
0,A,50990b,50990a
1,P,27382pr0005,27382
2,P,3298pb010,3298
3,P,19729pr0013a,19729
4,P,3070bpb043,3070b


In [28]:
parts = df_dic['parts']
parts.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32944 entries, 0 to 32943
Data columns (total 4 columns):
part_num            32944 non-null object
name                32944 non-null object
part_cat_id         32944 non-null int64
part_material_id    32944 non-null int64
dtypes: int64(2), object(2)
memory usage: 1.0+ MB


In [29]:
parts.head()

,part_num,name,part_cat_id,part_material_id
0,004591,Sticker Sheet 1 for Set 1650-1,58,1
1,004602,Sticker Sheet 2 for Set 1650-1,58,1
2,0687b1,Set 0687 Activity Booklet 1,17,1
3,0901,Baseplate 16 x 30 with Set 080 Yellow House Print,1,1
4,0902,Baseplate 16 x 24 with Set 080 Small White Hou...,1,1


#### merge parts

In [30]:
# parts + part_cat
parts2 = parts.merge(part_cat, how='left',left_on='part_cat_id',right_on='id')
parts2.head()

,part_num,name_x,part_cat_id,part_material_id,id,name_y
0,004591,Sticker Sheet 1 for Set 1650-1,58,1,58,Stickers
1,004602,Sticker Sheet 2 for Set 1650-1,58,1,58,Stickers
2,0687b1,Set 0687 Activity Booklet 1,17,1,17,Non-LEGO
3,0901,Baseplate 16 x 30 with Set 080 Yellow House Print,1,1,1,Baseplates
4,0902,Baseplate 16 x 24 with Set 080 Small White Hou...,1,1,1,Baseplates


In [31]:
parts2.columns = ['part_num', 'part_name', 'part_cat_id', 'part_material_id', 'id', 'part_cat_name']

In [32]:
parts2 = parts2.drop("id",axis=1)

In [80]:
parts2.to_csv("parts_with_cat.csv",index=False)

In [35]:
parts2.shape

(32944, 5)

### Price

In [36]:
price = df_dic['price_adj']
price.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17220 entries, 0 to 17219
Data columns (total 11 columns):
name             17220 non-null object
price            17220 non-null object
set_ID           17220 non-null object
rating           17030 non-null float64
price_USD        17220 non-null float64
year             17220 non-null int64
Index_19         17220 non-null float64
price_adjust     17220 non-null float64
pcs_num          15743 non-null float64
price_per_pcs    15743 non-null float64
ppp_nominal      15743 non-null float64
dtypes: float64(7), int64(1), object(3)
memory usage: 1.4+ MB


In [37]:
price.head()

,name,price,set_ID,rating,price_USD,year,Index_19,price_adjust,pcs_num,price_per_pcs,ppp_nominal
0,2 X 8 Bricks,"9,$0.50 ,5.6c,Box,Retail,Normal","""215""",4.0,0.5,1961,8.494824,4.25,9.0,0.47,0.06
1,4 x 4 Corner Bricks,"10,$0.50 ,5.0c,Box,Retail,Normal","""217""",4.0,0.5,1961,8.494824,4.25,10.0,0.42,0.05
2,2 x 4 Bricks,"20,$0.50 ,2.5c,Box,Retail,Normal","""218""",4.0,0.5,1961,8.494824,4.25,20.0,0.21,0.03
3,2 X 3 Bricks,"24,$0.50 ,2.1c,Box,Retail,Normal","""219""",4.0,0.5,1961,8.494824,4.25,24.0,0.18,0.02
4,2 X 2 Bricks,"33,$0.50 ,1.5c,Box,Retail,Normal","""220""",4.0,0.5,1961,8.494824,4.25,33.0,0.13,0.02


In [38]:
price = price.drop("price",axis=1)

### Sets

In [39]:
sets = df_dic['sets']
sets.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14815 entries, 0 to 14814
Data columns (total 5 columns):
set_num      14815 non-null object
name         14815 non-null object
year         14815 non-null int64
theme_id     14815 non-null int64
num_parts    14815 non-null int64
dtypes: int64(3), object(2)
memory usage: 578.8+ KB


In [40]:
sets.set_num = sets.set_num.apply(set_idchange)

In [41]:
sets.head()

,set_num,name,year,theme_id,num_parts
0,"""001""",Gears,1965,1,43
1,"""0011""",Town Mini-Figures,1978,84,12
2,"""0011""",Castle 2 for 1 Bonus Offer,1987,199,2
3,"""0012""",Space Mini-Figures,1979,143,12
4,"""0013""",Space Mini-Figures,1979,143,12


In [42]:
sets.year = sets.year.astype(str)
price.year = price.year.astype(str)

#### Join sets

In [43]:
# join price and rating
set_p = sets.merge(price.drop("name",axis=1),left_on=["set_num","year"], right_on=["set_ID","year"],how="left")

In [44]:
set_p = set_p.drop("set_ID",axis=1)
set_p.head()

,set_num,name,year,theme_id,num_parts,rating,price_USD,Index_19,price_adjust,pcs_num,price_per_pcs,ppp_nominal
0,"""001""",Gears,1965,1,43,NaN,4.95,8.056544,39.88,43.0,0.93,0.12
1,"""0011""",Town Mini-Figures,1978,84,12,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,"""0011""",Castle 2 for 1 Bonus Offer,1987,199,2,5.0,1.50,2.235670,3.35,151.0,0.02,0.01
3,"""0012""",Space Mini-Figures,1979,143,12,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,"""0013""",Space Mini-Figures,1979,143,12,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [51]:
set_pth = set_p.merge(themes,left_on="theme_id",right_on="id",how="left")

In [52]:
set_pth.columns

Index(['set_num', 'name_x', 'year', 'theme_id', 'num_parts', 'rating',
       'price_USD', 'Index_19', 'price_adjust', 'pcs_num', 'price_per_pcs',
       'ppp_nominal', 'id', 'name_y', 'parent_id'],
      dtype='object')

In [53]:
set_pth = set_pth.drop("id",axis=1)
set_pth.columns = ['set_num', 'set_name', 'year', 'theme_id', 'num_parts', 'rating',
                   'price_USD', 'Index_19', 'price_adjust', 'pcs_num', 'price_per_pcs',
                   'ppp_nominal','theme_name', 'parent_id']
set_pth.head()

,set_num,set_name,year,theme_id,num_parts,rating,price_USD,Index_19,price_adjust,pcs_num,price_per_pcs,ppp_nominal,theme_name,parent_id
0,"""001""",Gears,1965,1,43,NaN,4.95,8.056544,39.88,43.0,0.93,0.12,Technic,NaN
1,"""0011""",Town Mini-Figures,1978,84,12,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Supplemental,67.0
2,"""0011""",Castle 2 for 1 Bonus Offer,1987,199,2,5.0,1.50,2.235670,3.35,151.0,0.02,0.01,Lion Knights,186.0
3,"""0012""",Space Mini-Figures,1979,143,12,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Supplemental,126.0
4,"""0013""",Space Mini-Figures,1979,143,12,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Supplemental,126.0


In [54]:
set_pth2 = set_pth.merge(themes.drop("parent_id",axis=1),left_on="parent_id",right_on="id",how="left")

In [55]:
set_pth2.head()

,set_num,set_name,year,theme_id,num_parts,rating,price_USD,Index_19,price_adjust,pcs_num,price_per_pcs,ppp_nominal,theme_name,parent_id,id,name
0,"""001""",Gears,1965,1,43,NaN,4.95,8.056544,39.88,43.0,0.93,0.12,Technic,NaN,NaN,NaN
1,"""0011""",Town Mini-Figures,1978,84,12,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Supplemental,67.0,67.0,Classic Town
2,"""0011""",Castle 2 for 1 Bonus Offer,1987,199,2,5.0,1.50,2.235670,3.35,151.0,0.02,0.01,Lion Knights,186.0,186.0,Castle
3,"""0012""",Space Mini-Figures,1979,143,12,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Supplemental,126.0,126.0,Space
4,"""0013""",Space Mini-Figures,1979,143,12,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Supplemental,126.0,126.0,Space


In [56]:
set_pth2.name = set_pth2.name.fillna(set_pth2.theme_name)

In [57]:
set_pth2.columns

Index(['set_num', 'set_name', 'year', 'theme_id', 'num_parts', 'rating',
       'price_USD', 'Index_19', 'price_adjust', 'pcs_num', 'price_per_pcs',
       'ppp_nominal', 'theme_name', 'parent_id', 'id', 'name'],
      dtype='object')

In [58]:
set_pth2 = set_pth2.drop(['price_USD',"Index_19","ppp_nominal","parent_id","id"],axis=1)
set_pth2.columns = ['set_num', 'set_name', 'year', 'theme_id', 'num_parts', 'rating',
                    'price_adjust', 'pcs_num', 'price_per_pcs', 'theme_name', 'parent_theme']
set_pth2.head()

,set_num,set_name,year,theme_id,num_parts,rating,price_adjust,pcs_num,price_per_pcs,theme_name,parent_theme
0,"""001""",Gears,1965,1,43,NaN,39.88,43.0,0.93,Technic,Technic
1,"""0011""",Town Mini-Figures,1978,84,12,NaN,NaN,NaN,NaN,Supplemental,Classic Town
2,"""0011""",Castle 2 for 1 Bonus Offer,1987,199,2,5.0,3.35,151.0,0.02,Lion Knights,Castle
3,"""0012""",Space Mini-Figures,1979,143,12,NaN,NaN,NaN,NaN,Supplemental,Space
4,"""0013""",Space Mini-Figures,1979,143,12,NaN,NaN,NaN,NaN,Supplemental,Space


In [59]:
set_pth2.to_csv("sets_all.csv",index=False)

### themes

In [46]:
themes = df_dic['themes']
themes.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 671 entries, 0 to 670
Data columns (total 3 columns):
id           671 non-null int64
name         671 non-null object
parent_id    550 non-null float64
dtypes: float64(1), int64(1), object(1)
memory usage: 15.8+ KB


In [47]:
themes.head()

,id,name,parent_id
0,1,Technic,NaN
1,2,Arctic Technic,1.0
2,3,Competition,1.0
3,4,Expert Builder,1.0
4,5,Model,1.0


In [48]:
len(themes.parent_id.unique())

80

In [49]:
themes = themes.fillna(themes["id"])

In [50]:
themes.head()

,id,name,parent_id
0,1,Technic,NaN
1,2,Arctic Technic,1.0
2,3,Competition,1.0
3,4,Expert Builder,1.0
4,5,Model,1.0
